In [ ]:
from wholeslidedata.image.wholeslideimage import WholeSlideImage
from matplotlib import pyplot as plt

import ipywidgets as widgets
from IPython.display import display, HTML

import numpy as np
import os
from tqdm import tqdm

from py.helpers import BARRET_ROOT
from py.biopsy_detection import get_micron2pixel_fn, get_scanned_areas_from_wsi, \
    get_biopsies_from_scanned_area
from py.biopsy_detect_UI import get_slide_at_idx, plot_clean, get_contoured_img, \
    make_color_by_idx
from py.xml_utils import AnnotationGroup, dict_to_xml, map_contour_to_slide_coordinates, \
    add_contour_to_xml_dict
import os

os.add_dll_directory(r'C:\Program Files\openslide-win64\bin') # for openslide

In [ ]:
LANS_DIR = os.path.join(BARRET_ROOT, 'LANS_001-923')
LANS_BIOP_ROOT = os.path.join(BARRET_ROOT, 'p53_experiment_luuk_biopsy-level_no-HE')
LANS_BIOP_DIR = os.path.join(LANS_BIOP_ROOT, 'P53_score_high_consensus')
BIOPSY_DETECT_TEST_DIR = os.path.join(BARRET_ROOT, "Luuk_biopsy-detection", "test_data")

# Enter the target directory here:

In [ ]:
TARGET_DIR = os.path.join(BARRET_ROOT, "Luuk_biopsy-detection", "test_data")

MANUAL_CONFIRMATION = True

In [ ]:
spacing = 8
process_spacing = 64

slidepaths = [os.path.join(TARGET_DIR, f) for f in os.listdir(TARGET_DIR) if f.endswith('.tiff')]
print(f"Slides found: {len(slidepaths)}")

# Create XMLs and detect tissue
without manual confirmation

In [ ]:
if not MANUAL_CONFIRMATION:
    for slidepath in tqdm(slidepaths):
        try:
            wsi = WholeSlideImage(slidepath)
        except Exception as e:
            print(e)
            continue

        xml_path = slidepath.replace('.tiff', '.xml')
        if os.path.exists(xml_path):
            print(f"Skipping {xml_path} as file already exists")
            continue

        # Get areas
        m2p = get_micron2pixel_fn(spacing=spacing)
        scanned_areas, img, mask_closed, area_contours, area_boxes = get_scanned_areas_from_wsi(wsi, spacing=spacing,
            process_spacing=process_spacing, cut_patch_margin=0)
    
        xml_dict = None
        for area_idx in range(len(scanned_areas)):
            # Get biopsies
            scanned_area = scanned_areas[area_idx]
            contours, boxes, indices, filtered_indices, mask = get_biopsies_from_scanned_area(scanned_area, 
                                                                    m2p, min_size=300)
            # Save xml
            for biopsy_idx in range(len(indices)):
                mapped_contour = map_contour_to_slide_coordinates(contours[indices[biopsy_idx]], spacing, process_spacing, area_boxes[area_idx])
                group = "None"
                xml_dict = add_contour_to_xml_dict(xml_dict, mapped_contour, annotation_group=group)
                dict_to_xml(xml_dict).write(xml_path)


# Manual Confirmation Interface

In [ ]:
slide_output = widgets.Output()
biopsy_output = widgets.Output()
message_output = widgets.Output()

# To refresh with new slide
idx = 0
def refresh_slide():
    global idx, slidepath, m2p, scanned_areas, img, area_contours, area_boxes
    wsi, slidepath = get_slide_at_idx(idx, root_dir=TARGET_DIR)

    if not wsi:
        refresh_slide_idx(1)
        return
    global xml_dict, xml_path
    xml_path = slidepath.replace('.tiff', '.xml')
    xml_dict = None   

    m2p = get_micron2pixel_fn(spacing=spacing)
    scanned_areas, img, mask_closed, area_contours, area_boxes = get_scanned_areas_from_wsi(wsi, spacing=spacing,
        process_spacing=process_spacing, cut_patch_margin=0)
    refresh_area()

def refresh_slide_idx(idx_change=1):
    global idx
    idx += idx_change
    if idx < 0:
        idx = len(slidepaths) - 1
    elif idx >= len(slidepaths):
        idx = 0
    go_to_idx.value = idx
    # refresh_slide() # This is called by go_to_idx_changed

open_slide_btn = widgets.Button(description="Open slide outside notebook",
                            layout=widgets.Layout(width='auto', height='auto'))
def open_slide(_):
    global slidepath
    os.startfile(slidepath)
open_slide_btn.on_click(open_slide)

open_folder_btn = widgets.Button(description="Open folder outside notebook",
                            layout=widgets.Layout(width='auto', height='auto'))
def open_folder(_):
    global slidepath
    os.startfile(os.path.dirname(slidepath))
open_folder_btn.on_click(open_folder)

# To refresh with new area
area_idx = 0
def refresh_area(prev_change=1):
    global area_idx, scanned_area, contours, boxes, indices, filtered_indices, mask, area_contours, area_boxes, color_by_idx, slide_output, biopsy_output
    slide_output.clear_output()
    biopsy_output.clear_output()
    
    if area_idx == -1:
        area_idx = len(scanned_areas) - 1
    
    with slide_output:
        # plt.figure(figsize=(10, 2))
        fig, ax = plt.subplots(1, 1, figsize=(15, 2))
        print("Slide index: ", idx)
        plot_clean(get_contoured_img(img.copy(), area_contours, area_boxes, 3,
                                color_by_idx={area_idx: {'contour': (0, 255, 0), 'box': (255, 255, 255)}}
                                ), ax=ax)
        ax.set_title("Wholeslide\n(green = selected area)")
        plt.show()
    scanned_area = scanned_areas[area_idx]
    contours, boxes, indices, filtered_indices, mask = get_biopsies_from_scanned_area(scanned_area, 
                                                            m2p, min_size=300)
    
    if len(indices) == 0:
        refresh_area_idx(prev_change)
        return

    color_by_idx = make_color_by_idx(*filtered_indices)
    refresh_biopsy()

def refresh_area_idx(idx_change=1):
    global idx, area_idx, scanned_areas
    area_idx += idx_change
    if area_idx < 0:
        refresh_slide_idx(-1)
    elif area_idx >= len(scanned_areas):
        area_idx = 0
        refresh_slide_idx(1)
    else:
        refresh_area(prev_change=idx_change)


# To refresh with new biopsy
biopsy_idx = 0
def refresh_biopsy():
    global biopsy_idx, indices, color_by_idx, scanned_area, contours, boxes, mask, biopsy_output
    biopsy_output.clear_output()

    with biopsy_output:
        fig, ax = plt.subplots(1, 2, figsize=(20, 5))

        cbi = color_by_idx.copy()
        if len(indices) > 0:
            if biopsy_idx == -1:
                biopsy_idx = len(indices) - 1
            contour_idx = indices[biopsy_idx]
            cbi.update({contour_idx: {'contour': (0, 255, 0), 'box': (0, 0, 0)}})
            # Use numpy to get the bounding box of the contour from the scanned area
            box = boxes[contour_idx] # (x, y, w, h) where (x, y) is the top left corner
            biopsy_img = scanned_area[box[1]:box[1]+box[3], box[0]:box[0]+box[2]]
            plot_clean(biopsy_img, ax=ax[1])
            ax[1].set_title("selected biopsy zoomed")

        contoured = get_contoured_img(scanned_area.copy(), contours, boxes, 2, color_by_idx=cbi)
        # Place scanned_area and contoured next to each other in the first subplot using np.concatenate, with a black line in between
        show_mask = np.stack([mask, mask, mask], axis=2) * 255
        grid = np.concatenate((scanned_area, np.zeros((scanned_area.shape[0], 3, 3), dtype=np.uint8), 
                               show_mask, np.zeros((scanned_area.shape[0], 3, 3), dtype=np.uint8),
                               contoured), axis=1)
        plot_clean(grid, ax=ax[0])
        ax[0].set_title(f"area index: {area_idx}\nselected biopsy index: {biopsy_idx}\nscan  |  mask  |  contours")
        plt.show()

    global xml_dict, xml_path
    message_output.clear_output()
    with message_output:
        if os.path.exists(xml_path) and not xml_dict:
            display(HTML('<p style="color: red;">XML file exists, clicking "not biopsy" or "confirm biopsy" will overwrite it.</p>'))

def refresh_biopsy_idx(idx_change=1):
    global biopsy_idx, area_idx, indices
    biopsy_idx += idx_change
    if biopsy_idx < 0:
        refresh_area_idx(-1)
    elif biopsy_idx >= len(indices):
        biopsy_idx = 0
        refresh_area_idx(1)
    else:
        refresh_biopsy()


next_btn = widgets.Button(description="Next biopsy  >")
next_btn.on_click(lambda _: refresh_biopsy_idx(1))
prev_btn = widgets.Button(description="<  Previous biopsy")
prev_btn.on_click(lambda _: refresh_biopsy_idx(-1))

go_to_idx = widgets.IntText(value=0, description='Go to slide index:', layout=widgets.Layout(width='auto', height='auto'))
def go_to_idx_changed(change):
    if change['name'] != 'value':
        return
    global idx, area_idx, biopsy_idx
    idx = change['new']
    area_idx = 0
    biopsy_idx = 0
    refresh_slide()
go_to_idx.observe(go_to_idx_changed)


def save_annotation(is_biopsy):
    global xml_dict, xml_path, biopsy_idx, area_idx, area_boxes, indices
    if not xml_dict and (biopsy_idx > 0 or area_idx > 0):
        with message_output:
            display(HTML(f'<p style="color: orange;">Please start from biopsy and area index 0. (You are currently at biopsy index {biopsy_idx} and area index {area_idx}, click "< Previous biopsy" until you are at 0)</p>'))
        return
    mapped_contour = map_contour_to_slide_coordinates(contours[indices[biopsy_idx]], spacing, process_spacing, area_boxes[area_idx])
    group = "None" if is_biopsy else AnnotationGroup.EXCLUDE
    xml_dict = add_contour_to_xml_dict(xml_dict, mapped_contour, annotation_group=group)
    dict_to_xml(xml_dict).write(xml_path)
    refresh_biopsy_idx(1)


not_biopsy_btn = widgets.Button(description="Not biopsy")
not_biopsy_btn.on_click(lambda _: save_annotation(False))
yes_biopsy_btn = widgets.Button(description="Confirm biopsy")
yes_biopsy_btn.on_click(lambda _: save_annotation(True))

# Display
display(go_to_idx)
display(widgets.HBox([slide_output, open_slide_btn, open_folder_btn]))
display(widgets.HBox([prev_btn, not_biopsy_btn, yes_biopsy_btn, next_btn]), message_output, biopsy_output)
refresh_slide()